In [1]:
import numpy as np
import scipy.sparse as sp
import gurobipy as gp
from gurobipy import *

## Jobs to be scheduled 

In [2]:
#Jobs = ["Job1", "Job2", "Job3", "Job4", "Job5", "Job6", "Job7" , "Job8"]
jobs = ["Job1", "Job2", "Job3"]

## Constants

In [3]:
timeInMinutes = 12

## Time slots available 

In [4]:
timeSlots = [str(x) for x in range(timeInMinutes)]


Details of each job including release time, processing time, usage and deadline

In [5]:
jobs_details = { "Job1": { "r": 1, "t": 4, "u": 6,"d": 12 },
                 "Job2": { "r": 4, "t": 3, "u": 5,"d": 10},
                 "Job3": { "r": 2, "t": 5, "u": 7,"d": 12}
                 #"Job4": { "r": 0.5, "t": 0.7, "u": 2,"d": 0.2},
                 #"Job5": { "r": 0.5, "t": 0.7, "u": 3,"d": 0.2},
                 #"Job6": { "r": 0.5, "t": 0.7, "u": 5,"d": 0.2},
                 #"Job7": { "r": 0.5, "t": 0.7, "u": 6,"d": 0.2},
                 #"Job8": { "r": 0.5, "t": 0.7, "u": 7,"d": 0.2}
               }


## Predecessor of each job

In [6]:
jobs_predessors = { "Job1": [],
                 "Job2": [],
                 "Job3": ["Job1", "Job2"]
                 #"Job4": [],
                 #"Job5": ["Job4" , "Job2"],
                 #"Job6": [],
                 #"Job7": ["Job6"],
                 #"Job8": []
               }

# Model Generation

In [7]:
model = Model("Job Scheduling")

Using license file C:\Users\wel\gurobi.lic
Academic license - for non-commercial use only


## Creating dictionary/variables to be filled

In [8]:
maximumLoad =  model.addVar(vtype=GRB.CONTINUOUS, name= "Maximum Load")

In [9]:
decisionVariable = model.addVars(jobs, timeSlots, name="Decision Variable" , vtype = GRB.BINARY)


In [10]:
startTime = {}
for job in jobs:
        startTime[job] = model.addVar(vtype=GRB.INTEGER, name= f'Start Time {job}')


In [11]:
finishTime = {}
for job in jobs:
        finishTime[job] = model.addVar(vtype=GRB.INTEGER, name= f'Finish Time {job}')


In [12]:
totalLoad = {}
for time in timeSlots:
        totalLoad[time] = model.addVar(vtype=GRB.CONTINUOUS, name= f'Total load at time {time}')

## Adding constraints 

#### Atomocity Constraint
Each job should be run only once
$$\underset{t}\sum x_i(t) = 1 \;\;\;\; \forall i$$

In [13]:
model.addConstrs((quicksum(decisionVariable[job ,time] for time in timeSlots ) == 1 for job in jobs), name = " Atomocity Constraint")

{'Job1': <gurobi.Constr *Awaiting Model Update*>,
 'Job2': <gurobi.Constr *Awaiting Model Update*>,
 'Job3': <gurobi.Constr *Awaiting Model Update*>}

#### Start time of each job
Start time of the $j_i$ is t where $x_i$ is $1$ 
$$s_i = \underset{t}\sum t.x_i(t)$$

In [14]:
model.addConstrs(( startTime[job] ==  quicksum(decisionVariable[job ,time] * time for time in timeSlots)  for job in jobs) , name ="Start Time")

{'Job1': <gurobi.Constr *Awaiting Model Update*>,
 'Job2': <gurobi.Constr *Awaiting Model Update*>,
 'Job3': <gurobi.Constr *Awaiting Model Update*>}

$$s_i \geq r_i$$

In [15]:
model.addConstrs((startTime[job] >= jobs_details[job]["r"]  for job in jobs), name ="Start Time Limit")

{'Job1': <gurobi.Constr *Awaiting Model Update*>,
 'Job2': <gurobi.Constr *Awaiting Model Update*>,
 'Job3': <gurobi.Constr *Awaiting Model Update*>}

$$s_i + t_i \leq d_i$$

In [16]:
model.addConstrs((startTime[job] + jobs_details[job]["t"] <= jobs_details[job]["d"] for job in jobs), name ="Deadline Limit")

{'Job1': <gurobi.Constr *Awaiting Model Update*>,
 'Job2': <gurobi.Constr *Awaiting Model Update*>,
 'Job3': <gurobi.Constr *Awaiting Model Update*>}

$$f_i = s_i + t_i$$

In [17]:
model.addConstrs(( finishTime[job]  == (jobs_details[job]["t"] + startTime[job])  for job in jobs), name ="Finish Limit")

{'Job1': <gurobi.Constr *Awaiting Model Update*>,
 'Job2': <gurobi.Constr *Awaiting Model Update*>,
 'Job3': <gurobi.Constr *Awaiting Model Update*>}

#### Total load at time t
$$U_t = \underset{i}\sum(u_i*\underset{t-t_{i} < t^{'} \leq t }\sum  x_i(t^{'})) $$ 

In [18]:
model.addConstrs(( totalLoad[time]  == quicksum(jobs_details[job]["u"] * quicksum(decisionVariable[job,t] for t in timeSlots if ( float(t) > (float(time) - jobs_details[job]['t']) and float(t) <= float(time) )) for job in jobs ) for time in timeSlots ), name = "Total Load at time t")

{'0': <gurobi.Constr *Awaiting Model Update*>,
 '1': <gurobi.Constr *Awaiting Model Update*>,
 '2': <gurobi.Constr *Awaiting Model Update*>,
 '3': <gurobi.Constr *Awaiting Model Update*>,
 '4': <gurobi.Constr *Awaiting Model Update*>,
 '5': <gurobi.Constr *Awaiting Model Update*>,
 '6': <gurobi.Constr *Awaiting Model Update*>,
 '7': <gurobi.Constr *Awaiting Model Update*>,
 '8': <gurobi.Constr *Awaiting Model Update*>,
 '9': <gurobi.Constr *Awaiting Model Update*>,
 '10': <gurobi.Constr *Awaiting Model Update*>,
 '11': <gurobi.Constr *Awaiting Model Update*>}

#### Dependency Constraint

A job should not start before its predecessor finishes

$$ s_j + t_j \leq s_i \;\;\;\;  \forall j \;of \;p_i^{in} $$

In [19]:
for job in jobs:
    model.addConstrs(((startTime[predes]+ jobs_details[predes]["t"] <= startTime[job]) for predes in jobs_predessors[job]), name = "Dependency Constraint")

#### Maximum load
$$(\underset{t}{max} \; U_t)$$

In [20]:
model.addConstrs(( maximumLoad >= totalLoad[time] for time in  timeSlots), name = "Maximum Load")

{'0': <gurobi.Constr *Awaiting Model Update*>,
 '1': <gurobi.Constr *Awaiting Model Update*>,
 '2': <gurobi.Constr *Awaiting Model Update*>,
 '3': <gurobi.Constr *Awaiting Model Update*>,
 '4': <gurobi.Constr *Awaiting Model Update*>,
 '5': <gurobi.Constr *Awaiting Model Update*>,
 '6': <gurobi.Constr *Awaiting Model Update*>,
 '7': <gurobi.Constr *Awaiting Model Update*>,
 '8': <gurobi.Constr *Awaiting Model Update*>,
 '9': <gurobi.Constr *Awaiting Model Update*>,
 '10': <gurobi.Constr *Awaiting Model Update*>,
 '11': <gurobi.Constr *Awaiting Model Update*>}

### Objective Function

$$min \; (\underset{t}{max} \; U_t)$$

In [21]:
obj = maximumLoad

In [22]:
model.setObjective(obj, GRB.MINIMIZE)

In [23]:
model.optimize()

Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 41 rows, 55 columns and 249 nonzeros
Model fingerprint: 0x6ce19c90
Variable types: 13 continuous, 42 integer (36 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Presolve removed 41 rows and 55 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 11 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.100000000000e+01, best bound 1.100000000000e+01, gap 0.0000%


In [24]:
output = "<h1>Jobs Schedule</h1><table><tr><td></td><td><b>Start Time</b></td><td><b>FinishTime</b>"

for job in jobs:

    output += "<tr><td><b>{}</b></td><td style='text-align: right'>".format(job)
    
    # Start Time
    output += "<b>{:.1f}</b><br/>".format( startTime[job].X)
    
    # finish Time
    output += "</td><td style='text-align: right'>"
    output += "<b>{:.1f}</b><br/>".format( finishTime[job].X)
    
    output += "</td></tr>"
    
output += "</table>"


from IPython.display import HTML, display
display(HTML(output))

In [25]:
topBar = "<h1>Total Load at time t</h1><table><tr>"
for t in timeSlots:
    topBar = topBar + "<td></td><td><b>{0:s}</b>".format(t)
output = topBar + "<td></tr>"

# Load
lowerBar = "<tr>"
for t in timeSlots:
     lowerBar += "<td></td><td>{0:.1f}".format(totalLoad[t].X)
    #output += "<td style='text-align: right'>"
    #output += "<b>{:.1f}</b><br/>".format( totalLoad[t].X)

    
output =  output + lowerBar + "</tr>" 
output += "</table>"


from IPython.display import HTML, display
display(HTML(output))

,0,1,2,3,4,5,6,7,8,9,10
,0.0,6.0,6.0,6.0,11.0,5.0,5.0,7.0,7.0,7.0,7.0
